# Analysis of the Solow Model

Imports and set magics:

In [23]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**The Basic Solow Model** 

For our model project, we look into the dymamics of the Solow model. The Solow model, in its basic form, is a long-run economic model, which attempts to explain economic growth by capital accumulation. 

The Solow model with technological growth consists of the following functions:

$$ Y_t = K_t^\alpha(A_tL_t)^{1-\alpha}, 0 < \alpha < 1 $$
$$ K_{t+1}=S_t+(1-\delta)K_t,  0 < \delta < 1 $$
$$ S_t = sY_t,  0 < s < 1 $$
$$ L_{t+1}= (1+n)L_t $$
$$ A_{t+1} = (1+g)A_t $$

where
$Y_t$ is production, $K_t$ is physical capital, $A_t$ is technology, $S_t$ is savings and $L_t$ is labor.

To start analyzing the model, we start by defining the paramters beneath


In [24]:
# We define the parameters used in the model
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
k = sm.symbols('k')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')

## Analytical solution

We wish to find the transition equation for capital per effective worker, which we define as $ \tilde{k}_t=\frac{K_t}{A_tL_t}$.

Start by dividing capital accumulation with $ A_{t+1}L_{t+1}$

$$ \frac{K_{t+1}}{A_{t+1}L_{t+1}} = \frac{S_t+(1-\delta)K_t}{A_{t+1}L_{t+1}} $$

Then the last three functions described above for the model are inserted

$$ \tilde{k}_{t+1}=\frac{sY_t+(1-\delta)K_t}{L_t(1+n)A_t(1+g)} \Leftrightarrow $$
$$ \tilde{k}_{t+1}=\frac{1}{(1+n)(1+g)}[s\tilde{y_t}+(1-\delta)\tilde{k}_t]$$

Using the defintion for the tilde production function, $\tilde{y}_t=\frac{Y_t}{A_tL_t} $, we get $\tilde{y}_t=\tilde{k}_t^\alpha$. Thereby, the transition equation becomes

$$ \tilde{k}_{t+1}=\frac{1}{(1+n)(1+g)}[s\tilde{k}_t^\alpha+(1-\delta)\tilde{k}_t]$$

from which we find the analytical steady state below.

In [25]:
ss = sm.Eq(k,(s*k**alpha+(1-delta)*k)/((1+n)*(1+g)))
kss = sm.solve(ss,k)[0]
kss

((delta + g*n + g + n)/s)**(1/(alpha - 1))

## Numerical solution

Now we wish to solve the model for a numerical solution for the steady state. We use following parameter values:

$
\alpha = 1/3, \delta = 0.1, s = 0.2, g = 0.02, n = 0.01$


In [26]:
ss_func = sm.lambdify((s,g,n,delta,alpha),kss)

# Evaluate function
ss_func(0.2,0.02,0.01,0.1,1/3)

1.903831539231319

# Further analysis

We now decide to extend our Solow model to include human capital as well. The extended model will be defined by the following functions:

$$ Y_t=K_t^\alpha H_t^\phi (A_tL_t)^{1-\alpha-\sigma}, 0 < \alpha,\phi < 1 $$
$$ L_{t+1}=(1+n)L_t $$
$$ A_{t+1}(1+g)A_t$$
$$ K_{t+1}=s_KY_t+(1-\delta)K_t $$
$$ H_{t+1}=s_HY_t+(1-\delta)H_t $$

Compared to the variables in the basic Solow moodel, we now include $H_t$ for human capital.

As previously, we compute the per effective worker (tilde), functions

$$ \tilde{k}_{t+1}=\frac{1}{(1+n)(1+g)}[s_K\tilde{y}_t+(1-\delta)\tilde{k}_t ]$$

$$ \tilde{h}_{t+1}=\frac{1}{(1+n)(1+g)}[s_H\tilde{y}_t+(1-\delta)\tilde{h}_t ]$$

where $\tilde{y}_t=\tilde{k}_t^\alpha\tilde{h}_t^\phi $

In [27]:
# We define the parameters used in the model
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
phi = sm.symbols('phi')
k = sm.symbols('k')
h = sm.symbols('h')
s_K = sm.symbols('s_K')
s_H = sm.symbols('s_H')
g = sm.symbols('g')
n = sm.symbols('n')

And solve for the analytical steady state for physical and human capital, respectively:

In [32]:
y = k**alpha*h**phi

# For physical capital
ss1 = sm.Eq(k,(s_K*y+(1-delta)*k)/((1+n)*(1+g)))
kss1 = sm.solve(ss1,k)[0]
kss1

((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1))

In [33]:
# For human capital
ss2 = sm.Eq(h,(s_H*y+(1-delta)*h)/((1+n)*(1+g)))
hss = sm.solve(ss2,h)[0]
hss

((delta + g*n + g + n)/(k**alpha*s_H))**(1/(phi - 1))

Which we solve as two equations with two unknowns and get the following steady states:

$$\tilde{k}^*=\left(\frac{s_K^{1-\phi}s_H^\phi}{n + g + ng +\delta}\right)^{\frac{1}{1-\alpha-\phi}} $$

$$\tilde{h}^*=\left(\frac{s_K^{\alpha}s_H^{1-\alpha}}{n + g + ng +\delta}\right)^{\frac{1}{1-\alpha-\phi}} $$

We also wish to compute the numerical values, which is done below. We use following parameters:

$
\alpha = 1/3, \phi = 1/3, \delta = 0.1, s_K = 0.2, s_H = 0.1, g = 0.02, n = 0.01$


In [42]:
# We define the two steady states:
k_tilde = ((s_K**(1-phi) * s_H**phi)/(n+g+delta +n*g))**(1/(1-phi-alpha))
h_tilde = ( (s_K**(alpha) * s_H**(1-alpha))/(n+g+delta +n*g))**(1/(1-phi-alpha))
        
# Now we turn them in to python function, using sympy lambdify.
kss_function = sm.lambdify((alpha,phi,delta,n,g,s_K,s_H),k_tilde)
hss_function = sm.lambdify((alpha,phi,delta,n,g,s_K,s_H),h_tilde) 

print(f'The steady state value for k is {kss_function(1/3,1/3,0.1,0.01,0.02,0.2,0.1):1f}, and the steady state value for h is {hss_function(1/3,1/3,0.1,0.01,0.02,0.2,0.1):4f}')

The steady state value for k is 1.812287, and the steady state value for h is 0.906144


# Conclusion

Add concise conclusion. 